# 推論のためのPipeline

## Pipelineの使い方

タスク固有の`pipeline()`を使用する代わりに，全てのタスク固有のパイプラインを含む一般的な`pipeline()`を使用するのがより簡単です．

`pipeline()`は自動的にデフォルトのモデルと前処理クラスを読み込みます．

1. `pipeline()`を作成し，推論タスクを指定して始めます:

In [6]:
from transformers import pipeline

generator = pipeline(task="automatic-speech-recognition")

No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 55bb623 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.
/home/fujie/.conda/envs/py311/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT 

2. `pipeline()`に入力データを渡します:

In [7]:
generator("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

{'text': 'I HAVE A DREAM BUT ONE DAY THIS NATION WILL RISE UP LIVE UP THE TRUE MEANING OF ITS TREES'}

`task`を指定することで，デフォルトのモデルと前処理クラスが自動的に読み込まれます．

`model`を指定することで，指定したモデルが読み込まれます．
より大規模な音声認識モデルを使用してみましょう．

In [8]:
generator = pipeline(model="openai/whisper-large")
generator("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'text': ' I have a dream that one day this nation will rise up and live out the true meaning of its creed.'}

## パラメータ

`pipeline()`は多くのパラメータをサポートしています．
パラメータにはタスク固有のものと，全パイプラインに共通のものがあります．
`pipeline()`のインスタンス作成時や，推論時にパラメータを指定することができます．
推論時に指定したパラメータはインスタンス作成時のパラメータよりも優先されますが，一時的なものとしてのみ使用されます．


```python
generator = pipeline(modeul="openai/whisper-large", my_parameter=1)
out = generator(...) # このとき，my_parameter=1が使用されます
out = generator(..., my_parameter=2) # このとき，my_parameter=2が使用されます
out = genetrator(...) # このとき，my_parameter=1が使用されます
```

### デバイス

`device=n`を指定すると，パイプラインはモデルを指定したデバイスに自動的に配置します．

```python
generator = pipeline(model="openai/whisper-large", device=0) # GPU 0にモデルを配置します
```

モデルが単一のGPUには大きすぎる場合，`device_map="auto"`を設定して，
`Accelerate`にモデルの重みをどのようにロードし，保存するかを自動的に決定させることができます．

```python
generator = pipeline(model="opeanai/whisper-large", device_map="auto")
```

`device_map="auto"`にしたときは，`device`パラメータは指定すべきではありません．

### バッチサイズ

デフォルトではパイプラインはバッチ処理をしませんが，`batch_size=n`を指定すると，バッチ処理を行うことができます．

```python
generator = pipeline(model="openai/whisper-large", batch_size=8)
audio_files = [f"audio_{i}.wav" for i in range(16)]
texts = generator(audio_files)
```

`texts`にはバッチ処理の有無に関わらず同様の結果が得られます．